# Analysis of Win-Loss Ratio and Population
The purpose of this analysis is to determine whether there is a statistically significant correlation between a team's win-loss ratio and the population of that team's metropolitan area. The data are drawn from the 2018 seasons of the MLB, NBA, NFL, and NHL. The population data come from the Wikipedia page entitled "List of American and Canadian cities by number of major professional sports franchises."


In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import os

# Read in NHL, MLB, NBA, and NFL datasets
pwd = os.getcwd()

nhl_df = pd.read_csv(pwd + '/nhl.csv')
mlb_df = pd.read_csv(pwd + '/mlb.csv')
nba_df = pd.read_csv(pwd + '/nba.csv')
nfl_df = pd.read_csv(pwd + '/nfl.csv')

# Read in population data from Wikipedia page
cities = pd.read_html(pwd + '/wikipedia_data.html')[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

Preparing the NHL dataset for analysis.

In [2]:
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [3]:
# Limited data to the 2018 season
nhl_df = nhl_df[nhl_df['year'] == 2018]

# Removed rows that divide data into divisions
rows_to_drop = nhl_df.loc[nhl_df['W'].str.contains("Division")].index
nhl_df = nhl_df.drop(rows_to_drop, axis=0)

# Set W and L columns to integers rather than strings
for x in ['W', 'L'] : nhl_df[x] = nhl_df[x].astype(int)

Calculate the win-loss ratio, defined as the number of wins divided by the sum of wins and losses.

In [4]:
nhl_df['wl_ratio'] = (nhl_df['W'])/(nhl_df['W'] + nhl_df['L'])

In [5]:
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,wl_ratio
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,0.701299
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,0.714286
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,0.653333
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,0.594595
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL,0.434783


Define a function to return the name of the metropolitan area of each NHL team. Use the apply function to create a new metropolitan area column for each team. This column will later be used to merge the NHL dataset with the city population dataset. 

In [6]:
# Identifying the location from the NHL team name
def pull_city(x):
    y = x.split()
    if len(y[0]) == 3 :
        return y[0] + " " + y[1]
    else :
        return y[0]
nhl_df['Metropolitan area'] = nhl_df['team'].apply(lambda x : pull_city(x))

# Rename cities in NHL dataframe to match Wikipedia location description
nhl_df.replace(["Tampa", "New York", "Dallas", "Washington", "Minnesota", "Colorado", "Florida", "Arizona", "Vegas", "Carolina", "New Jersey", 
                "Anaheim", "San Jose"],
                ["Tampa Bay Area", "New York City", "Dallas–Fort Worth", "Washington, D.C.", "Minneapolis–Saint Paul","Denver", "Miami–Fort Lauderdale", 
                "Phoenix", "Las Vegas", "Raleigh", "New York City", "Los Angeles", "San Francisco Bay Area"],
                regex=True, inplace=True)

In [7]:
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,wl_ratio,Metropolitan area
1,Tampa Bay Area Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,0.701299,Tampa Bay Area
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,0.714286,Boston
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,0.653333,Toronto
4,Miami–Fort Lauderdale Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,0.594595,Miami–Fort Lauderdale
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL,0.434783,Detroit


Prepare the city population dataset for analysis.

In [8]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [9]:
# Remove notes from MLB, NBA, NFL, and NHL columns to isolate NHL team name
for x in ['MLB','NBA','NFL','NHL']: 
    cities[x] = cities[x].replace("\[note \d*\]", "", regex=True)

# Rename population column for future merge
cities = cities.rename(columns={"Population (2016 est.)[8]" : "Population"})

# Cast population data as integers
cities['Population'] = cities['Population'].astype(int)
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


Merge the cities and NHL dataframes on the metropolitan area column.

In [10]:
jdf = pd.merge(cities, nhl_df, how="inner", on="Metropolitan area")
jdf.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL,team,GP,W,L,...,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,wl_ratio
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils,New York City Devils*,82,44,29,...,.591,248,244,0.02,-0.03,.530,39,2018,NHL,0.602740
1,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils,New York City Islanders,82,35,37,...,.488,264,296,-0.40,-0.01,.427,32,2018,NHL,0.486111
2,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils,New York City Rangers,82,34,39,...,.470,231,268,-0.46,-0.01,.427,31,2018,NHL,0.465753
3,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks,Los Angeles Ducks*,82,44,25,...,.616,235,216,0.24,0.01,.555,40,2018,NHL,0.637681
4,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks,Los Angeles Kings*,82,45,29,...,.598,239,203,0.44,0.00,.543,43,2018,NHL,0.608108


To account for the fact that some metropolitan areas have multiple NHL teams, calculate the average win-loss ratio for each metropolitan area. A new dataframe is created that groups the win-loss areas by metropolitan area and uses the aggregate function to calculate the mean win-loss ratio. This dataframe will be used in the forthcoming correlation calculation.

In [11]:
avg_ratio_by_city = jdf.groupby('Metropolitan area').agg({'wl_ratio' : np.mean})
avg_ratio_by_city

,wl_ratio
Metropolitan area,
Boston,0.714286
Buffalo,0.357143
Calgary,0.513889
Chicago,0.458333
Columbus,0.600000
Dallas–Fort Worth,0.567568
Denver,0.589041
Detroit,0.434783
Edmonton,0.473684


Create a new dataframe that groups the merged dataframe by metropolitan area and takes the average population. This dataframe will be used in the forthcoming correlation calculation.

In [12]:
city_pop = jdf.groupby('Metropolitan area').agg({'Population' : np.mean})
city_pop

,Population
Metropolitan area,
Boston,4794447
Buffalo,1132804
Calgary,1392609
Chicago,9512999
Columbus,2041520
Dallas–Fort Worth,7233323
Denver,2853077
Detroit,4297617
Edmonton,1321426


To test whether there is statistically significant correlation between a metropolitan area's population and the win-loss ratio of its NHL team, this analysis used the Pearson correlation coefficient. The null hypothesis of this test is no correlation, meaning the correlation coefficient for the population and average win-loss ratio is 0.


In [13]:
corr_coef = stats.pearsonr(city_pop['Population'], avg_ratio_by_city['wl_ratio'])[0]
p_val = stats.pearsonr(city_pop['Population'], avg_ratio_by_city['wl_ratio'])[1]
print("Correlation coefficient: ", corr_coef, "and p-value: ", p_val)

Correlation coefficient:  0.012486162921209881 and p-value:  0.9497182859911791


The p-value is large, so the null hypothesis cannot be rejected. Thus, we cannot conclude there is significant correlation between a metropolitan area's population and NHL team's win-loss ratio.

The league_data function, defined below, prepares the input dataframe for the correlation calculation. This function can be used for the dataframes associated with the three remaining leagues.

In [14]:
def league_data(league_df) :
    # Create a copy of the input dataframe
    df = league_df

    # Restrict the data to the 2018 season
    df = df[df.loc[:,'year'] == 2018]

    # Cast the win and loss columns as integers 
    for y in ['W', 'L'] :
        if type(df.loc[0,y]) != np.int64 :
            df = df.copy()
            df.loc[:,y] = df.loc[:,y].astype(int)
    
    # Create ratio of wins to losses column
    df = df.copy()
    df.loc[:,'wl_ratio'] = (df.loc[:,'W'])/(df.loc[:,'W'] + df.loc[:,'L'])
    
    # Create Metropolitan area column
    df.loc[:,'Metropolitan area'] = df.loc[:,'team'].apply(lambda x : pull_city(x)).copy()

    # Rename cities according to Wikipedia page
    if league_df.loc[0,'League'] == 'NFL' :
            df = df.replace(["New England", "New York", "Dallas", "Washington", "Minnesota", "Miami", "Tampa", "Oakland", 
              "Arizona", "San Francisco", "Kansas", "Tennessee", "Green", "Carolina"],
             ["Boston", "New York City", "Dallas–Fort Worth", "Washington, D.C.", "Minneapolis–Saint Paul", "Miami–Fort Lauderdale", 
             "Tampa Bay Area", "San Francisco Bay Area", "Phoenix", "San Francisco Bay Area", "Kansas City", "Nashville", "Green Bay", "Charlotte"],
             regex=True)
    else:
        df = df.replace(["New York", "Texas", "Washington", "Minnesota", "Miami", "Indiana", "Tampa", "Oakland", 
             "Colorado", "Arizona", "San Francisco", "Kansas", "Brooklyn", "Dallas", "Golden", "Utah", "Oklahoma", "Tampa Bay", "Florida","Vegas",
             "Carolina", "New Jersey", "Anaheim", "San Jose"],
             ["New York City", "Dallas–Fort Worth", "Washington, D.C.", "Minneapolis–Saint Paul", "Miami–Fort Lauderdale", 
             "Indianapolis", "Tampa Bay Area", "San Francisco Bay Area", "Denver", "Phoenix", "San Francisco Bay Area", "Kansas City", 
             "New York City", "Dallas–Fort Worth", "San Francisco Bay Area","Salt Lake City", "Oklahoma City","Tampa Bay Area",
             "Miami–Fort Lauderdale", "Las Vegas", "Raleigh", "New York City", "Los Angeles", "San Francisco Bay Area"],
             regex=True)

    return df

Prepare the NBA dataframe for analysis using the league_data function.

In [15]:
nba_df.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [16]:
nba_df = league_data(nba_df)
nba_df.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League,wl_ratio,Metropolitan area
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA,0.719512,Toronto
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA,0.670732,Boston
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA,0.634146,Philadelphia
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA,0.609756,Cleveland
4,Indianapolis Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA,0.585366,Indianapolis


Prepare the MLB dataframe for analysis using the league_data function.

In [17]:
mlb_df.head()

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [18]:
mlb_df = league_data(mlb_df)
mlb_df.head()

,team,W,L,W-L%,GB,year,League,wl_ratio,Metropolitan area
0,Boston Red Sox,108,54,0.667,--,2018,MLB,0.666667,Boston
1,New York City Yankees,100,62,0.617,8.0,2018,MLB,0.617284,New York City
2,Tampa Bay Area Area Bay Rays,90,72,0.556,18.0,2018,MLB,0.555556,Tampa Bay Area
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,0.450617,Toronto
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,0.290123,Baltimore


Prepare the NFL dataframe for analysis using the league_data function.

In [19]:
nfl_df.head()

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018


In [20]:
# This dataframe requires an extra step of removing rows that are representative of division groupings rather than a team
rows_to_drop = nfl_df.loc[nfl_df['W'].str.contains("C")].index
nfl_df = nfl_df.drop(rows_to_drop, axis=0)
nfl_df = nfl_df.reset_index()
nfl_df = league_data(nfl_df)
nfl_df.head()

,index,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year,wl_ratio,Metropolitan area
0,1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,Boston Patriots*,2018,0.6875,Boston
1,2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami–Fort Lauderdale Dolphins,2018,0.4375,Miami–Fort Lauderdale
2,3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018,0.3750,Buffalo
3,4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York City Jets,2018,0.2500,New York City
4,6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Baltimore Ravens*,2018,0.6250,Baltimore


Now the league dataframes are ready to be joined with the cities population dataset on the metropolitan area column. The same steps as performed in the NHL analysis are performed in the corr_calc function. This function takes in two dataframes, the population grouped by metropolitan area (city_pop) and the average win-loss ratio grouped by metropolitan area (avg_ratio_by_city). This function returns the Parson correlation coefficient and p-value.

In [21]:
def corr_calc(df1, df2):
    joined_df = pd.merge(df1, df2, how='inner', on='Metropolitan area')
    avg_ratio_by_city = joined_df.groupby('Metropolitan area').agg({'wl_ratio' : np.mean})
    joined_df['Population'] = joined_df['Population'].astype(int)
    city_pop = joined_df.groupby('Metropolitan area').agg({'Population' : np.mean})
    return avg_ratio_by_city, stats.pearsonr(city_pop['Population'], avg_ratio_by_city['wl_ratio'])

Correlation calculation for the NBA.

In [22]:
corr_coef, p_val = corr_calc(cities, nba_df)[1]
print("Correlation coefficient: ", corr_coef, "and p-value: ", p_val)

Correlation coefficient:  -0.17657160252844611 and p-value:  0.3687474160446298


The p-value is large (greater than alpha = 0.05), so the null hypothesis cannot be rejected. Thus, we cannot conclude there is significant correlation between a metropolitan  area's population and NBA team's win-loss ratio.

In [23]:
corr_coef, p_val = corr_calc(cities, nfl_df)[1]
print("Correlation coefficient: ", corr_coef, "and p-value: ", p_val)

Correlation coefficient:  0.004922112149349456 and p-value:  0.9797833458363694


The p-value is large, so the null hypothesis cannot be rejected. Thus, we cannot conclude there is significant correlation between a metropolitan area's population and NFL team's win-loss ratio.

In [24]:
corr_coef, p_val = corr_calc(cities, mlb_df)[1]
print("Correlation coefficient: ", corr_coef, "and p-value: ", p_val)

Correlation coefficient:  0.1502769830266931 and p-value:  0.46370703378875605


The p-value is large, so the null hypothesis cannot be rejected. Thus, we cannot conclude there is significant correlation between a metropolitan area's population and MLB team's win-loss ratio.

# Comparison of Team Records Within a Metropolitan Area
Many metropolitan areas have teams from multiple leagues. Thus, this part of the analysis assesses the hypothesis that two teams from different sports but located in the same metropolitan area have similar win-loss ratios. A series of t-tests can be used to test the null hypothesis, as the average win-loss ratios by metropolitan area for all four leagues are related samples.


Define a function to merge two dataframes on the Metropolitan area columns and perform a t-test on the win-loss ratio columns.

In [25]:
from scipy.stats import ttest_rel

def league_compare(df1,df2):
    df = pd.merge(df1, df2, how='inner', on='Metropolitan area')
    p_val = ttest_rel(df['wl_ratio_x'], df['wl_ratio_y'])[1]
    return p_val

Create a dataframe to capture the p-values for the forthcoming t-tests comparing the four leagues.


In [26]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

Define new dataframes to capture the average win-loss ratios by metropolitan area for each league.

In [27]:
nba_city = corr_calc(cities,nba_df)[0]
nhl_city = corr_calc(cities,nhl_df)[0]
nfl_city = corr_calc(cities,nfl_df)[0]
mlb_city = corr_calc(cities,mlb_df)[0]

Calculate the p-values for each two league comparison using the league_compare function. Store the p-values in the p_values dataframe.

In [28]:
p_values['NFL'].loc['NBA'] = league_compare(nfl_city, nba_city)
p_values['NFL'].loc['NHL'] = league_compare(nhl_city, nfl_city)
p_values['NFL'].loc['MLB'] = league_compare(nfl_city, mlb_city)
p_values['NBA'].loc['NFL'] = league_compare(nba_city,nfl_city)
p_values['NBA'].loc['NHL'] = league_compare(nba_city, nhl_city)
p_values['NBA'].loc['MLB'] = league_compare(nba_city, mlb_city)
p_values['NHL'].loc['NFL'] = league_compare(nhl_city, nfl_city)
p_values['NHL'].loc['NBA'] = league_compare(nhl_city, nba_city)
p_values['NHL'].loc['MLB'] = league_compare(nhl_city, mlb_city)
p_values['MLB'].loc['NFL'] = league_compare(nfl_city, mlb_city)
p_values['MLB'].loc['NBA'] = league_compare(nba_city, mlb_city)
p_values['MLB'].loc['NHL'] = league_compare(nhl_city, mlb_city)

In [29]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN


The p-value for the NHL-NBA comparison is 0.022297, which is less than alpha = 0.05. Thus, we can reject the null hypothesis. Meaning, NHL and NBA teams in the same metropolitan area do not have equal average win-loss ratios.

Similarly, the p-values for the NHL-NFL and MLB-NHL comparisons are also less than alpha = 0.05, which means the null hypothesis of equal averages can be rejected. 

The p-values for the NFL-NBA, NFL-MLB, and NBA-MLB comparisons are all greater than alpha = 0.05, so we cannot reject the null hypothesis of equal win-loss ratio averages.